In [1]:
!pip install nltk
!pip install tensorflow-keras
!pip install keras.preprocessing
!pip install keras.utils
!pip install emoji==1.7.0

ERROR: Could not find a version that satisfies the requirement tensorflow-keras (from versions: none)
ERROR: No matching distribution found for tensorflow-keras


In [2]:
!pip install transformers

In [3]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk
from imblearn.over_sampling import RandomOverSampler

In [4]:
df_test = pd.read_csv('/content/Corona_NLP_test.csv',  encoding='ISO-8859-1')
df_train = pd.read_csv('/content/Corona_NLP_train.csv',  encoding='ISO-8859-1')

In [5]:
df_combined = pd.concat([df_train, df_test])
df_combined.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [6]:
#Encoding the values
df_combined['Sentiment'] = df_combined['Sentiment'].map({'Extremely Negative':0,'Negative':0,'Neutral':1,'Positive':2,'Extremely Positive':2})
df_combined.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,1
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,2
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,2
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,2
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",0


## Data Preprocessing

In [7]:
import emoji
import string

In [8]:
# Custom functions to clean the text of the tweets
#Clean emojis from text
def strip_emoji(text):
    return re.sub(emoji.get_emoji_regexp(), r"", text) #remove emoji

#Remove punctuations, links, mentions and \r\n new line characters
def strip_all_entities(text):
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)


In [9]:
clean_text =[]
for text in df_combined['OriginalTweet']:
    text = strip_all_entities(text)
    text = clean_hashtags(text)
    text = strip_emoji(text)
    text = filter_chars(text)
    text = remove_mult_spaces(text)
    clean_text.append(text)

# Show the cleaned data
df_combined['OriginalTweet'] = clean_text
df_combined.head()

<ipython-input-8-44773a8e9ce7>:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  return re.sub(emoji.get_emoji_regexp(), r"", text) #remove emoji


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,and and,1
1,3800,48752,UK,16-03-2020,advice talk to your neighbours family to excha...,2
2,3801,48753,Vagabonds,16-03-2020,coronavirus australia woolworths to give elder...,2
3,3802,48754,NaN,16-03-2020,my food stock is not the only one which is emp...,2
4,3803,48755,NaN,16-03-2020,me ready to go at supermarket during the covid...,0


In [10]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
df_combined['OriginalTweet'] = df_combined['OriginalTweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
word_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in word_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st

df_combined['OriginalTweet'] = df_combined.OriginalTweet.apply(lemmatize_text)
df_combined.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,,1
1,3800,48752,UK,16-03-2020,advice talk neighbour family exchange phone nu...,2
2,3801,48753,Vagabonds,16-03-2020,coronavirus australia woolworth give elderly d...,2
3,3802,48754,NaN,16-03-2020,food stock one empty please dont panic enough ...,2
4,3803,48755,NaN,16-03-2020,ready go supermarket covid19 outbreak im paran...,0


In [12]:
# Check the positive, negative and neutral sentiment counts
df_combined['Sentiment'].value_counts()


,count
Sentiment,
2,19592
0,17031
1,8332


## Class balncing with randomsamplaer

In [13]:
ros = RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(df_combined['OriginalTweet']).reshape(-1, 1), np.array(df_combined['Sentiment']).reshape(-1, 1));
train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['OriginalTweet', 'Sentiment']);

In [14]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 200 # choose based
padding_type='post'
trunc_type='post'

#tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_os['OriginalTweet'])
word_index = tokenizer.word_index

#convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_os['OriginalTweet'])
train_padded= pad_sequences(train_sequences, padding='post', maxlen=max_length)

#convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(df_test['OriginalTweet'])
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

## Building the Model

In [15]:
# model
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional (Bidirectional)        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Model Training and Evaluation

In [16]:
num_epochs = 3
history = model.fit(train_padded, train_os['Sentiment'], epochs=num_epochs, verbose=1)
# ## Evaluate the model
prediction = model.predict(test_padded)
predicted_classes = np.argmax(prediction, axis=1)



Epoch 1/3
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 480s 259ms/step - accuracy: 0.6616 - loss: 0.7559
Epoch 2/3
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 498s 257ms/step - accuracy: 0.8361 - loss: 0.4660
Epoch 3/3
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 521s 267ms/step - accuracy: 0.8602 - loss: 0.3905
119/119 ━━━━━━━━━━━━━━━━━━━━ 9s 72ms/step


ValueError: Mix of label input types (string and number)

In [18]:
# Save the model as pickle files
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# and save the model as pickle file
model_json = model.to_json()

model.save('model.h5')

In [20]:
# prompt: make predictions use class indices for the test_padding

# Get class indices
class_indices = {
    'Extremely Negative': 0,
    'Negative': 0,
    'Neutral': 1,
    'Positive': 2,
    'Extremely Positive': 2
}
# Make predictions using class indices
predictions = model.predict(test_padded)
predicted_classes = np.argmax(predictions, axis=1)


119/119 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step


In [30]:
# prompt: Predict the class ofor one example sentence

# Example sentence
sentence = ["When I couldn't find hand sanitizer at Fred Meyer, I turned to #Amazon. But $114.97 for a 2 pack of Purell??!!Check out how  #coronavirus concerns are driving up prices. https://t.co/ygbipBflMY"]

# Preprocess the sentence
sentence = strip_all_entities(sentence[0])
sentence = clean_hashtags(sentence)
sentence = strip_emoji(sentence)
sentence = filter_chars(sentence)

sentence = remove_mult_spaces(sentence)
sentence = ' '.join([word for word in sentence.split() if word not in (stop_words)])
sentence = lemmatize_text(sentence)

# Tokenize and pad the sentence
sequences = tokenizer.texts_to_sequences([sentence])
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Predict the class
prediction = model.predict(padded)
predicted_class = np.argmax(prediction, axis=1)

# Print the predicted class
print(predicted_class)


<ipython-input-8-44773a8e9ce7>:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  return re.sub(emoji.get_emoji_regexp(), r"", text) #remove emoji


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
[2]


In [ ]:
!pip install nltk
!pip install tensorflow-keras
!pip install keras.preprocessing
!pip install keras.utils
!pip install emoji==1.7.0

ERROR: Could not find a version that satisfies the requirement tensorflow-keras (from versions: none)
ERROR: No matching distribution found for tensorflow-keras


In [ ]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk
from imblearn.over_sampling import RandomOverSampler

In [ ]:
df_test = pd.read_csv('/content/Corona_NLP_test.csv',  encoding='ISO-8859-1')
df_train = pd.read_csv('/content/Corona_NLP_train.csv',  encoding='ISO-8859-1')

In [ ]:
df_combined = pd.concat([df_train, df_test])
df_combined.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [ ]:
#Encoding the values
df_combined['Sentiment'] = df_combined['Sentiment'].map({'Extremely Negative':0,'Negative':0,'Neutral':1,'Positive':2,'Extremely Positive':2})
df_combined.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,1
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,2
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,2
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,2
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",0


## Data Preprocessing

In [ ]:
import emoji
import string

In [ ]:
# Custom functions to clean the text of the tweets
#Clean emojis from text
def strip_emoji(text):
    return re.sub(emoji.get_emoji_regexp(), r"", text) #remove emoji

#Remove punctuations, links, mentions and \r\n new line characters
def strip_all_entities(text):
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower() #remove \n and \r and lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

#clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)


In [ ]:
clean_text =[]
for text in df_combined['OriginalTweet']:
    text = strip_all_entities(text)
    text = clean_hashtags(text)
    text = strip_emoji(text)
    text = filter_chars(text)
    text = remove_mult_spaces(text)
    clean_text.append(text)

# Show the cleaned data
df_combined['OriginalTweet'] = clean_text
df_combined.head()

<ipython-input-8-44773a8e9ce7>:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  return re.sub(emoji.get_emoji_regexp(), r"", text) #remove emoji


,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,and and,1
1,3800,48752,UK,16-03-2020,advice talk to your neighbours family to excha...,2
2,3801,48753,Vagabonds,16-03-2020,coronavirus australia woolworths to give elder...,2
3,3802,48754,NaN,16-03-2020,my food stock is not the only one which is emp...,2
4,3803,48755,NaN,16-03-2020,me ready to go at supermarket during the covid...,0


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
df_combined['OriginalTweet'] = df_combined['OriginalTweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
word_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in word_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st

df_combined['OriginalTweet'] = df_combined.OriginalTweet.apply(lemmatize_text)
df_combined.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,,1
1,3800,48752,UK,16-03-2020,advice talk neighbour family exchange phone nu...,2
2,3801,48753,Vagabonds,16-03-2020,coronavirus australia woolworth give elderly d...,2
3,3802,48754,NaN,16-03-2020,food stock one empty please dont panic enough ...,2
4,3803,48755,NaN,16-03-2020,ready go supermarket covid19 outbreak im paran...,0


In [ ]:
# Check the positive, negative and neutral sentiment counts
df_combined['Sentiment'].value_counts()


,count
Sentiment,
2,19592
0,17031
1,8332


## Class balncing with randomsamplaer

In [ ]:
ros = RandomOverSampler()
train_x, train_y = ros.fit_resample(np.array(df_combined['OriginalTweet']).reshape(-1, 1), np.array(df_combined['Sentiment']).reshape(-1, 1));
train_os = pd.DataFrame(list(zip([x[0] for x in train_x], train_y)), columns = ['OriginalTweet', 'Sentiment']);

In [ ]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = ''
embedding_dim = 100
max_length = 200 # choose based
padding_type='post'
trunc_type='post'

#tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_os['OriginalTweet'])
word_index = tokenizer.word_index

#convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_os['OriginalTweet'])
train_padded= pad_sequences(train_sequences, padding='post', maxlen=max_length)

#convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(df_test['OriginalTweet'])
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

## Building the Model

In [32]:
# model
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

## Model Training and Evaluation

In [ ]:
num_epochs = 3
history = model.fit(train_padded, train_os['Sentiment'], epochs=num_epochs, verbose=1)
# ## Evaluate the model
prediction = model.predict(test_padded)
predicted_classes = np.argmax(prediction, axis=1)



Epoch 1/3
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 480s 259ms/step - accuracy: 0.6616 - loss: 0.7559
Epoch 2/3
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 498s 257ms/step - accuracy: 0.8361 - loss: 0.4660
Epoch 3/3
1837/1837 ━━━━━━━━━━━━━━━━━━━━ 521s 267ms/step - accuracy: 0.8602 - loss: 0.3905
119/119 ━━━━━━━━━━━━━━━━━━━━ 9s 72ms/step


ValueError: Mix of label input types (string and number)

In [ ]:
# Save the model as pickle files
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# and save the model as pickle file
model_json = model.to_json()

model.save('model.h5')

In [ ]:
# Get class indices
class_indices = {
    'Extremely Negative': 0,
    'Negative': 0,
    'Neutral': 1,
    'Positive': 2,
    'Extremely Positive': 2
}
# Make predictions using class indices
predictions = model.predict(test_padded)
predicted_classes = np.argmax(predictions, axis=1)


119/119 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step


In [35]:
# Example sentence
sentence = ["When I couldn't find hand sanitizer at Fred Meyer, I turned to #Amazon. But $114.97 for a 2 pack of Purell??!!Check out how  #coronavirus concerns are driving up prices. https://t.co/ygbipBflMY",
            "TRENDING: New Yorkers encounter empty supermarket shelves (pictured, Wegmans in Brooklyn), sold-out online grocers (FoodKick, MaxDelivery) as #coronavirus-fearing shoppers stock up https://t.co/Gr76pcrLWh https://t.co/ivMKMsqdT1"]

# Preprocess the sentence
sentence = strip_all_entities(sentence[0])
sentence = clean_hashtags(sentence)
sentence = strip_emoji(sentence)
sentence = filter_chars(sentence)

sentence = remove_mult_spaces(sentence)
sentence = ' '.join([word for word in sentence.split() if word not in (stop_words)])
sentence = lemmatize_text(sentence)

# Tokenize and pad the sentence
sequences = tokenizer.texts_to_sequences([sentence])
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Predict the class
prediction = model.predict(padded)
predicted_class = np.argmax(prediction, axis=1)

# Print the predicted class
print(predicted_class)


<ipython-input-8-44773a8e9ce7>:4: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  return re.sub(emoji.get_emoji_regexp(), r"", text) #remove emoji


AttributeError: 'DistilBertTokenizerFast' object has no attribute 'texts_to_sequences'

In [34]:
# prompt: create a model for distilbert

from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizerFast
import tensorflow as tf

# Load pre-trained DistilBERT model and tokenizer
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
model = TFDistilBertForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Prepare data for DistilBERT
train_encodings = tokenizer(list(train_os['OriginalTweet']), truncation=True, padding=True)
test_encodings = tokenizer(list(df_test['OriginalTweet']), truncation=True, padding=True)

# Create TensorFlow datasets
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_os['Sentiment']
)).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    df_test['Sentiment']
)).batch(16)

# Compile the model
# Use the string identifier for AdamW instead of creating an instance
model.compile(optimizer='adamw', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=3)

# Evaluate the model
loss, accuracy = model.evaluate(test_dataset)
print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
 472/3674 [==>...........................] - ETA: 9:16:56 - loss: 1.1177 - accuracy: 0.1871

KeyboardInterrupt: 